In [1]:
import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
#import fem, sys, os, time, Bio.PDB,warnings,nglview
import sys, os, Bio.PDB, warnings
pdb_list = Bio.PDB.PDBList()
pdb_parser = Bio.PDB.PDBParser()
#data_dir = '../../protein_pfam'
data_dir = 'data'
sys.path.append(data_dir)
#from parse_pfam import parse_pfam
from parse_pfam_test import parse_pfam
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)
%matplotlib inline
#from IPython.display import Image
#from ipymol import viewer
#viewer.start()
#pymol = viewer._server

In [2]:
pfam, pdb_refs = parse_pfam(data_dir)
print 'total MSAs: %i, total PDB refs: %i' % (pfam.shape[0], pdb_refs.shape[0])

total MSAs: 6, total PDB refs: 285


In [3]:
#pfam['size'] = pfam['res'] * pfam['seq']
pfam.sort_values(by='seq', ascending=False, inplace=True)
pfam.head()

,res,seq,pdb_refs
PF10417,113,9746,555
PF12574,0,0,0
PF09847,0,0,0
PF00244,0,0,0
PF16998,0,0,0


In [4]:
# Cas 9 family accesion code
ac = 'PF10417'
# store the family of interest in the 'fam' variable
fam = pfam.loc[ac]
#print 'size rank: %i of %i' % (pfam['size'].rank(ascending=False)[fam.name].astype(int), pfam.shape[0])

In [5]:
# local directory containing data for this MSA
fam_dir = os.path.join(data_dir, 'Pfam-A.full', fam.name)

# the residue symbols array that is the MSA
msa = np.load(os.path.join(fam_dir, 'msa.npy'))

In [6]:
print(msa.shape)

(113, 9746)


In [7]:
print(msa[:,0])

['-' '-' '-' '-' '-' '-' '-' '-' '-' '-' 'a' '-' 'A' 'Q' 'F' 'V' 'A' 'E'
 'H' '-' '-' '-' 'G' 'D' '-' 'q' 'V' 'C' 'P' 'A' 'K' 'W' '-' 'T' 'P' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' 'G' 'A'
 'E' 'T' '-' 'I' 'V' '-' 'P' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' 's' 'l' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-']


In [8]:
print(msa[:,1])

['-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' 'A' 'F' 'Q' 'F' 'T' 'D' 'K'
 'H' '-' '-' '-' 'G' 'E' '-' '-' 'V' 'C' 'P' 'A' 'G' 'W' '-' 'K' 'P' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' 'G' 'S'
 'D' 'T' '-' 'I' '-' '-' '-' '-' '-' '-' 'K' '-' 'P' '-' 'D' '-' 'V' '-'
 'E' '-' '-' '-' 'K' '-' '-' '-' 'S' 'K' 'E' 'Y' 'F' 'S' 'K' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-']


In [9]:
print(msa[:,2])

['-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' 'S' 'L' 'Q' 'L' 'T' 'D' 'G'
 'Y' '-' '-' '-' 'K' 'V' '-' '-' 'A' 'T' 'P' 'V' 'N' 'W' '-' 'K' 'D' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' 'G' 'D'
 'D' 'V' '-' 'I' 'I' '-' 'V' '-' 'P' '-' 'S' '-' 'L' '-' 'Q' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' 'd' 'p' 'a'
 'e' 'l' 'a' 'q' 'r' 'f' 'p' 'k' 'g' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-']


## PDB structures

In [10]:
pdb_refs['res'] = pdb_refs['pdb_end'] - pdb_refs['pdb_start'] + 1
pdb_refs.sort_values(by='seq', ascending=False, inplace=True)
pdb_refs.head()

,seq,uniprot_id,uniprot_start,uniprot_end,pdb_id,chain,pdb_start,pdb_end,res
PF10417,8778,PRDX2_HUMAN,161,196,1QMV,F,161,196,36
PF10417,8778,PRDX2_HUMAN,161,196,1QMV,G,161,196,36
PF10417,8778,PRDX2_HUMAN,161,196,1QMV,A,161,196,36
PF10417,8778,PRDX2_HUMAN,161,196,1QMV,D,161,196,36
PF10417,8778,PRDX2_HUMAN,161,196,1QMV,C,161,196,36


In [11]:
refs = pdb_refs[pdb_refs.index.str.contains(fam.name)]
#refs.head()
print(refs.shape)
print(refs)

(285, 9)
          seq   uniprot_id  uniprot_start  uniprot_end pdb_id chain  \
PF10417  8778  PRDX2_HUMAN            161          196   1QMV     F   
PF10417  8778  PRDX2_HUMAN            161          196   1QMV     G   
PF10417  8778  PRDX2_HUMAN            161          196   1QMV     A   
PF10417  8778  PRDX2_HUMAN            161          196   1QMV     D   
PF10417  8778  PRDX2_HUMAN            161          196   1QMV     C   
PF10417  8778  PRDX2_HUMAN            161          196   1QMV     B   
PF10417  8778  PRDX2_HUMAN            161          196   1QMV     J   
PF10417  8778  PRDX2_HUMAN            161          196   1QMV     H   
PF10417  8778  PRDX2_HUMAN            161          196   1QMV     I   
PF10417  8778  PRDX2_HUMAN            161          196   1QMV     E   
PF10417  8449   TDXH_PYRHO            155          198   5XBQ     A   
PF10417  8449   TDXH_PYRHO            155          198   3W6G     H   
PF10417  8449   TDXH_PYRHO            155          198   3W6G     S 

In [12]:
i = 0  # from 0 to refs.shape[0]  (0-->19)
ref = refs.iloc[i]
print(ref)

seq                     8778
uniprot_id       PRDX2_HUMAN
uniprot_start            161
uniprot_end              196
pdb_id                  1QMV
chain                      F
pdb_start                161
pdb_end                  196
res                       36
Name: PF10417, dtype: object


In [13]:
pdb_file = pdb_list.retrieve_pdb_file(ref.pdb_id, pdir=fam_dir, file_format='pdb')

In [14]:
print(pdb_file)

data/Pfam-A.full/PF10417/pdb1qmv.ent


In [15]:
pdb_seq = pdb_parser.get_structure(ref.pdb_id, pdb_file)

In [17]:
#print(pbd_seq.shape)
print(pdb_seq)

<Structure id=1QMV>


In [20]:
print(pdb_seq[0])

<Model id=0>


In [21]:
chain = pdb_parser.get_structure(ref.pdb_id, pdb_file)[0][ref.chain]

In [22]:
print(chain)

<Chain id=F>


In [23]:
coords = np.array([a.get_coord() for a in chain.get_atoms()])

In [24]:
print(coords.shape)
print(coords)

(1770, 3)
[[ 23.177 -17.875  39.31 ]
 [ 21.837 -17.334  39.329]
 [ 21.852 -16.136  38.356]
 ...
 [ 47.407 -30.266  42.513]
 [ 49.833 -29.394  40.141]
 [ 53.746 -29.739  41.809]]


In [25]:
coords = coords[ref.pdb_start-1:ref.pdb_end]

In [26]:
print(coords.shape)
print(coords)

(36, 3)
[[  7.532 -24.46   12.827]
 [  6.695 -25.229  12.378]
 [  7.001 -22.19   11.947]
 [  7.389 -20.986  11.151]
 [  8.725 -20.611  11.014]
 [  6.412 -20.198  10.577]
 [  9.087 -19.494  10.255]
 [  6.752 -19.065   9.851]
 [  8.078 -18.73    9.692]
 [  8.053 -24.534  14.047]
 [  7.701 -25.623  14.957]
 [  7.72  -25.092  16.383]
 [  8.529 -24.207  16.708]
 [  8.745 -26.726  14.788]
 [  8.507 -28.012  15.55 ]
 [  9.631 -29.002  15.289]
 [  9.304 -30.375  15.864]
 [ 10.547 -31.171  16.084]
 [  6.882 -25.633  17.262]
 [  6.907 -25.206  18.664]
 [  7.979 -25.982  19.407]
 [  8.123 -27.172  19.148]
 [  5.545 -25.474  19.302]
 [  5.482 -25.117  20.789]
 [  4.174 -25.673  21.358]
 [  4.14  -26.875  21.691]
 [  3.223 -24.877  21.438]
 [  8.82  -25.289  20.185]
 [  9.898 -25.982  20.893]
 [  9.707 -25.705  22.383]
 [  9.404 -24.56   22.764]
 [ 11.303 -25.492  20.474]
 [ 12.369 -26.107  21.383]
 [ 11.665 -25.894  19.055]
 [  9.789 -26.75   23.221]
 [  9.602 -26.499  24.653]]
